In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import t as tstat
from statsmodels.tools.tools import *

from optimize import opt 
from sampler import *
from algorithms import *

In [5]:
def t_test(result_list, T, delta=0.1):
    trial = len(result_list)
        
    pval_list = []
    
    for i in range(trial):        
        result = np.array(result_list[i])

        idx =np.array([i+1 for i in range(len(result))])
        s = np.sqrt(result[:, 1]/idx)
        thetas = result[:, 0]
        
        tt = thetas/s
        
        DF = len(thetas) - 1
                
        pval_list.append(tstat.sf(np.abs(tt), DF)*2)
    
    return np.array(pval_list)

In [6]:
def rejection_or_not(pval_list, alpha, size=-250):
    trial = len(pval_list)
    
    reject = 0
    
    for i in range(trial):
        if pval_list[i, size] < alpha:
            reject += 1
    
    reject /= trial
    
    return reject

In [7]:
def multiple_testing(pval_list, alpha):
    trial = len(pval_list)
    
    reject = 0
    
    for i in range(trial):
        if pval_list[i, -350] < alpha:
            reject += 150
        elif pval_list[i, -250]*2 < alpha:
            reject += 250
        elif pval_list[i, -150]*3 < alpha:
            reject += 350
        elif pval_list[i, -50]*4 < alpha:
            reject += 450
        else:
            reject += 500
        
    reject /= trial
    
    return reject

In [8]:
def sequential_testing(result_list, T, delta=0.1):
    trial = len(result_list)
    
    stopping_time = 0
    
    for i in range(trial):
        result = np.array(result_list[i])

        idx =np.array([i+1 for i in range(len(result))])
        v = result[:, 2]*idx
        thetas = np.abs(result[:, 0]*idx)
        lil = 1.1*(np.log(1/delta) + np.sqrt(2*v*np.log(np.log(v)/delta)))

        stop_times = idx[(thetas > lil) == True]

        if len(stop_times) > 0:
            stopping_time += idx[(thetas > lil) == True][0]

        else:
            stopping_time += T
        
    stopping_time /= trial
    
    return stopping_time

In [9]:
mu0 = 0.3
mu1 = 0.8
std0 = 0.3
std1 = 0.8

trial = 1000
T = 500

mu = mu1 - mu0

In [ ]:
rct_res = []
a2ipw_knn_res = []
a2ipw_nw_res = []
ma2ipw_knn_res = []
ma2ipw_nw_res = []
adaipw_knn_res =  []
adaipw_nw_res = []
dm_knn_res = []
dm_nw_res = []
hahn50_knn_res =  []
hahn50_nw_res =  []
hahn100_knn_res =  []
hahn100_nw_res =  []
opt_res =  []

for t in range(trial):    
    rct = RCT()
    a2ipw_knn = A2IPW(method='NW', pretraining=50, gamma= lambda t: t**(-1/1.5))
    a2ipw_nw = A2IPW(method='NW', pretraining=50, gamma= lambda t: t**(-1))
    ma2ipw_knn = A2IPW(method='NW', pretraining=10, gamma= lambda t: t**(-1/1.5))
    ma2ipw_nw = A2IPW(method='NW', pretraining=10, gamma= lambda t: t**(-1))
    adaipw_knn = MA2IPW(method='NW', pretraining=50, gamma= lambda t: t**(-1/1.5), zeta=lambda t: t**(-1))
    adaipw_nw = MA2IPW(method='NW', pretraining=50, gamma= lambda t: t**(-1/1.5), zeta=lambda t: t**(-2))
    dm_knn = MA2IPW(method='NW', pretraining=50, gamma= lambda t: t**(-1), zeta=lambda t: t**(-1))
    dm_nw = MA2IPW(method='NW', pretraining=50, gamma= lambda t: t**(-1), zeta=lambda t: t**(-2))
    hahn50_knn = MA2IPW(method='NW', pretraining=10, gamma= lambda t: t**(-1/1.5), zeta=lambda t: t**(-1))
    hahn50_nw = MA2IPW(method='NW', pretraining=10, gamma= lambda t: t**(-1/1.5), zeta=lambda t: t**(-2))
    hahn100_knn = MA2IPW(method='NW', pretraining=10, gamma= lambda t: t**(-1), zeta=lambda t: t**(-1))
    hahn100_nw = MA2IPW(method='NW', pretraining=10, gamma= lambda t: t**(-1), zeta=lambda t: t**(-2))
    opt = OPT()

    rct_temp = []
    a2ipw_knn_temp = []
    a2ipw_nw_temp = []
    ma2ipw_knn_temp = []
    ma2ipw_nw_temp = []
    adaipw_knn_temp =  []
    adaipw_nw_temp = []
    dm_knn_temp = []
    dm_nw_temp = []
    hahn50_knn_temp =  []
    hahn50_nw_temp =  []
    hahn100_knn_temp =  []
    hahn100_nw_temp =  []
    opt_temp =  []

    for period_t in range(T):            
        X, Y0, Y1, EY0, EY1, Var0, Var1 = sampler(mu0, mu1, std0, std1, type_sampler='homo')
        
        rct(period_t, X, Y0, Y1)
        a2ipw_knn(period_t, X, Y0, Y1)
        a2ipw_nw(period_t, X, Y0, Y1)
        ma2ipw_knn(period_t, X, Y0, Y1)
        ma2ipw_nw(period_t, X, Y0, Y1)
        adaipw_knn(period_t, X, Y0, Y1)
        adaipw_nw(period_t, X, Y0, Y1)
        dm_knn(period_t, X, Y0, Y1)
        dm_nw(period_t, X, Y0, Y1)
        hahn50_knn(period_t, X, Y0, Y1)
        hahn50_nw(period_t, X, Y0, Y1)
        hahn100_knn(period_t, X, Y0, Y1)
        hahn100_nw(period_t, X, Y0, Y1)
        opt(period_t, X, Y0, Y1, EY0, EY1, Var0, Var1)

        if period_t > 2:
            rct_temp.append(rct.effect())
            a2ipw_knn_temp.append(a2ipw_knn.effect())
            a2ipw_nw_temp.append(a2ipw_nw.effect())
            ma2ipw_knn_temp.append(ma2ipw_knn.effect())
            ma2ipw_nw_temp.append(ma2ipw_nw.effect())
            adaipw_knn_temp.append(adaipw_knn.effect())
            adaipw_nw_temp.append(adaipw_nw.effect())
            dm_knn_temp.append(dm_knn.effect())
            dm_nw_temp.append(dm_nw.effect())
            hahn50_knn_temp.append(hahn50_knn.effect())
            hahn50_nw_temp.append(hahn50_nw.effect())
            hahn100_knn_temp.append(hahn100_knn.effect())
            hahn100_nw_temp.append(hahn100_nw.effect())
            opt_temp.append(opt.effect())

    rct_res.append(rct_temp)
    a2ipw_knn_res.append(a2ipw_knn_temp)
    a2ipw_nw_res.append(a2ipw_nw_temp)
    ma2ipw_knn_res.append(ma2ipw_knn_temp)
    ma2ipw_nw_res.append(ma2ipw_nw_temp)
    adaipw_knn_res.append(adaipw_knn_temp)
    adaipw_nw_res.append(adaipw_nw_temp)
    dm_knn_res.append(dm_knn_temp)
    dm_nw_res.append(dm_nw_temp)
    hahn50_knn_res.append(hahn50_knn_temp)
    hahn50_nw_res.append(hahn50_nw_temp)
    hahn100_knn_res.append(hahn100_knn_temp)
    hahn100_nw_res.append(hahn100_nw_temp)
    opt_res.append(opt_temp)
    
rct_res = np.array(rct_res)
a2ipw_knn_res = np.array(a2ipw_knn_res)
a2ipw_nw_res = np.array(a2ipw_nw_res)
ma2ipw_knn_res = np.array(ma2ipw_knn_res)
ma2ipw_nw_res = np.array(ma2ipw_nw_res)
adaipw_knn_res =  np.array(adaipw_knn_res)
adaipw_nw_res = np.array(adaipw_nw_res)
dm_knn_res = np.array(dm_knn_res)
dm_nw_res = np.array(dm_nw_res)
hahn50_knn_res =  np.array(hahn50_knn_res)
hahn50_nw_res =  np.array(hahn50_nw_res)
hahn100_knn_res =  np.array(hahn100_knn_res)
hahn100_nw_res =  np.array(hahn100_nw_res)
opt_res =  np.array(opt_res)

In [14]:
rct_res = np.array(rct_res)
a2ipw_knn_res = np.array(a2ipw_knn_res)
a2ipw_nw_res = np.array(a2ipw_nw_res)
ma2ipw_knn_res = np.array(ma2ipw_knn_res)
ma2ipw_nw_res = np.array(ma2ipw_nw_res)
adaipw_knn_res =  np.array(adaipw_knn_res)
adaipw_nw_res = np.array(adaipw_nw_res)
dm_knn_res = np.array(dm_knn_res)
dm_nw_res = np.array(dm_nw_res)
hahn50_knn_res =  np.array(hahn50_knn_res)
hahn50_nw_res =  np.array(hahn50_nw_res)
hahn100_knn_res =  np.array(hahn100_knn_res)
hahn100_nw_res =  np.array(hahn100_nw_res)
opt_res =  np.array(opt_res)

In [15]:
delta0 = 0.1

MSEs = np.zeros((14, T-3))
STDs = np.zeros((14, T-3))
tau_time = np.zeros(14)
bf_time = np.zeros(14)

rejection250_time = np.zeros(14)
rejection500_time = np.zeros(14)

MSEs[0] = np.mean((rct_res[:, :, 0] - mu)**2, axis=0)
MSEs[1] = np.mean((a2ipw_knn_res[:, :, 0] - mu)**2, axis=0)
MSEs[2] = np.mean((a2ipw_nw_res[:, :, 0] - mu)**2, axis=0)
MSEs[3] = np.mean((ma2ipw_knn_res[:, :, 0] - mu)**2, axis=0)
MSEs[4] = np.mean((ma2ipw_nw_res[:, :, 0] - mu)**2, axis=0)
MSEs[5] = np.mean((adaipw_knn_res[:, :, 0] - mu)**2, axis=0)
MSEs[6] = np.mean((adaipw_nw_res[:, :, 0] - mu)**2, axis=0)
MSEs[7] = np.mean((dm_knn_res[:, :, 0] - mu)**2, axis=0)
MSEs[8] = np.mean((dm_nw_res[:, :, 0] - mu)**2, axis=0)
MSEs[9] = np.mean((hahn50_knn_res[:, :, 0] - mu)**2, axis=0)
MSEs[10] = np.mean((hahn50_nw_res[:, :, 0] - mu)**2, axis=0)
MSEs[11] = np.mean((hahn100_knn_res[:, :, 0] - mu)**2, axis=0)
MSEs[12] = np.mean((hahn100_nw_res[:, :, 0] - mu)**2, axis=0)
MSEs[13] = np.mean((opt_res[:, :, 0] - mu)**2, axis=0)

STDs[0] = np.std((rct_res[:, :, 0] - mu)**2, axis=0)
STDs[1] = np.std((a2ipw_knn_res[:, :, 0] - mu)**2, axis=0)
STDs[2] = np.std((a2ipw_nw_res[:, :, 0] - mu)**2, axis=0)
STDs[3] = np.std((ma2ipw_knn_res[:, :, 0] - mu)**2, axis=0)
STDs[4] = np.std((ma2ipw_nw_res[:, :, 0] - mu)**2, axis=0)
STDs[5] = np.std((adaipw_knn_res[:, :, 0] - mu)**2, axis=0)
STDs[6] = np.std((adaipw_nw_res[:, :, 0] - mu)**2, axis=0)
STDs[7] = np.std((dm_knn_res[:, :, 0] - mu)**2, axis=0)
STDs[8] = np.std((dm_nw_res[:, :, 0] - mu)**2, axis=0)
STDs[9] = np.std((hahn50_knn_res[:, :, 0] - mu)**2, axis=0)
STDs[10] = np.std((hahn50_nw_res[:, :, 0] - mu)**2, axis=0)
STDs[11] = np.std((hahn100_knn_res[:, :, 0] - mu)**2, axis=0)
STDs[12] = np.std((hahn100_nw_res[:, :, 0] - mu)**2, axis=0)
STDs[13] = np.std((opt_res[:, :, 0] - mu)**2, axis=0)

tau_time[0] = sequential_testing(rct_res, T, delta=delta0)
tau_time[1] = sequential_testing(a2ipw_knn_res, T, delta=delta0)
tau_time[2] = sequential_testing(a2ipw_nw_res, T, delta=delta0)
tau_time[3] = sequential_testing(ma2ipw_knn_res, T, delta=delta0)
tau_time[4] = sequential_testing(ma2ipw_nw_res, T, delta=delta0)
tau_time[5] = sequential_testing(adaipw_knn_res, T, delta=delta0)
tau_time[6] = sequential_testing(adaipw_nw_res, T, delta=delta0)
tau_time[7] = sequential_testing(dm_knn_res, T, delta=delta0)
tau_time[8] = sequential_testing(dm_nw_res, T, delta=delta0)
tau_time[9] = sequential_testing(hahn50_knn_res, T, delta=delta0)
tau_time[10] = sequential_testing(hahn50_nw_res, T, delta=delta0)
tau_time[11] = sequential_testing(hahn100_knn_res, T, delta=delta0)
tau_time[12] = sequential_testing(hahn100_nw_res, T, delta=delta0)
tau_time[13] = sequential_testing(opt_res, T, delta=delta0)

pval_list = t_test(rct_res, T)
rejection250_time[0] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[0] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[0] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(a2ipw_knn_res, T)
rejection250_time[1] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[1] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[1] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(a2ipw_nw_res, T)
rejection250_time[2] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[2] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[2] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(ma2ipw_knn_res, T)
rejection250_time[3] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[3] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[3] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(ma2ipw_nw_res, T)
rejection250_time[4] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[4] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[4] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(adaipw_knn_res, T)
rejection250_time[5] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[5] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[5] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(adaipw_nw_res, T)
rejection250_time[6] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[6] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[6] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(dm_knn_res, T)
rejection250_time[7] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[7] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[7] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(dm_nw_res, T)
rejection250_time[8] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[8] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[8] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(hahn50_knn_res, T)
rejection250_time[9] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[9] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[9] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(hahn50_nw_res, T)
rejection250_time[10] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[10] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[10] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(hahn100_knn_res, T)
rejection250_time[11] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[11] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[11] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(hahn100_nw_res, T)
rejection250_time[12] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[12] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[12] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(opt_res, T)
rejection250_time[13] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[13] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[13] = multiple_testing(pval_list, alpha=0.05)

/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in log
  if sys.path[0] == '':
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in greater
  
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in greater
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in sqrt
  if sys.path[0] == '':


In [16]:
result_list = [MSEs[:, -350], STDs[:, -350], np.round(rejection250_time*100, 1), MSEs[:, -200], STDs[:, -200], np.round(rejection500_time*100, 1), np.round(tau_time, 1), np.round(bf_time, 1) ]


In [17]:
print(pd.DataFrame(np.round(result_list, 3)).T.to_latex())

\begin{tabular}{lrrrrrrrr}
\toprule
{} &      0 &      1 &      2 &      3 &      4 &      5 &      6 &      7 \\
\midrule
0  &  0.138 &  0.186 &   24.8 &  0.073 &  0.106 &   45.0 &  450.2 &  373.5 \\
1  &  0.060 &  0.090 &   52.4 &  0.025 &  0.035 &   87.2 &  306.2 &  239.6 \\
2  &  0.066 &  0.099 &   52.0 &  0.026 &  0.037 &   87.5 &  308.5 &  240.6 \\
3  &  0.069 &  0.095 &   48.1 &  0.028 &  0.038 &   86.6 &  321.6 &  252.2 \\
4  &  0.074 &  0.106 &   47.0 &  0.027 &  0.037 &   86.0 &  322.4 &  254.6 \\
5  &  0.063 &  0.092 &   53.3 &  0.024 &  0.035 &   89.9 &  301.4 &  235.4 \\
6  &  0.065 &  0.091 &   49.5 &  0.028 &  0.039 &   85.2 &  314.4 &  249.6 \\
7  &  0.069 &  0.096 &   49.1 &  0.027 &  0.038 &   88.0 &  312.1 &  246.9 \\
8  &  0.070 &  0.098 &   45.8 &  0.027 &  0.039 &   83.9 &  327.1 &  255.6 \\
9  &  0.075 &  0.114 &   46.6 &  0.028 &  0.043 &   85.0 &  321.2 &  254.0 \\
10 &  0.075 &  0.109 &   47.4 &  0.026 &  0.035 &   84.6 &  319.7 &  252.2 \\
11 &  0.144 &  0.18

In [70]:
result_list = [MSEs[:, -400], STDs[:, -400], MSEs[:, -300], STDs[:, -300], MSEs[:, -200], STDs[:, -200], MSEs[:, -100], STDs[:, -100], MSEs[:, -1], STDs[:, -1] ]


In [71]:
print(pd.DataFrame(np.round(result_list, 3)).T.to_latex())

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &      0 &      1 &      2 &      3 &      4 &      5 &      6 &      7 &      8 &      9 \\
\midrule
0  &  0.218 &  0.292 &  0.109 &  0.144 &  0.073 &  0.100 &  0.054 &  0.074 &  0.044 &  0.064 \\
1  &  0.132 &  0.184 &  0.062 &  0.085 &  0.038 &  0.054 &  0.029 &  0.039 &  0.024 &  0.033 \\
2  &  0.110 &  0.158 &  0.045 &  0.063 &  0.025 &  0.035 &  0.017 &  0.024 &  0.013 &  0.018 \\
3  &  0.139 &  0.185 &  0.068 &  0.103 &  0.044 &  0.058 &  0.033 &  0.047 &  0.025 &  0.038 \\
4  &  0.104 &  0.147 &  0.041 &  0.056 &  0.023 &  0.033 &  0.016 &  0.024 &  0.012 &  0.017 \\
5  &  0.218 &  0.293 &  0.111 &  0.167 &  0.075 &  0.103 &  0.055 &  0.078 &  0.043 &  0.061 \\
6  &  0.239 &  0.350 &  0.124 &  0.181 &  0.081 &  0.115 &  0.061 &  0.085 &  0.049 &  0.070 \\
7  &  0.259 &  0.368 &  0.130 &  0.190 &  0.086 &  0.126 &  0.064 &  0.094 &  0.051 &  0.075 \\
8  &  0.187 &  0.271 &  0.076 &  0.116 &  0.045 &  0.066 &  0.031 &  0.046 &  0.024 &  0.